# ROMS netcdf check
**Author: Jun Sasaki, Coded on August 23, 2020, Updated on August 24, 2020**<br>
A sample code for manipulating `Projects/Sandy/Sandy_clm.nc`

- Checking initial, boundary, and climatorology netcdf files for ROMS in COAWST based on [ROMS Ocean Model](http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html) Example.
- No specific s coordinate is defined in Sandy example; thus sigma coordinate (-1 to 0) is assumed.

In [ ]:
import numpy as np
from netCDF4 import num2date, date2num
import datetime
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import hvplot.xarray
import holoviews as hv
from geoviews import opts
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Reading grid netcdf file
- See [Coordinate system of (xi, eta)](https://www.myroms.org/wiki/Fractional_Coordinate_System_(%CE%BE_-_%CE%B7_space))

In [ ]:
grid_nc = './Sandy_roms_grid.nc'
#grid_nc = './training/Sandy_roms_grid.nc'

with xr.open_dataset(grid_nc) as grd:
    pass
grd

# Reading climatology netcdf file

In [ ]:
clm_nc = "./Sandy_clm.nc"
#clm_nc = "./training/merged_coawst_clm.nc"

with xr.open_dataset(clm_nc) as ds:
    pass

## Chunking and creating sigma coordinate

In [ ]:
### ここでのChunk（チャンク）は時系列データを先頭から一定の長さで分割した断片的なデータ集合
### ds.chunk({'salt_time':1}) は DataSetを salt_time について1タイムステップ毎に分割
ds = ds.chunk({'ocean_time':1})  ### Any time may be ok for chunking.

### Converting to sigma coordinate
ds_rho = 1.0/len(ds['s_rho'])
ds['s_rho']=-ds_rho/2.0 + ds['s_rho'] * (-ds_rho)

ds.coords['s_rho']=ds['s_rho']
ds.coords['erho']=ds['erho']
ds.coords['xrho']=ds['xrho']
ds.coords['lon_rho']=ds['lon_rho']
ds.coords['lat_rho']=ds['lat_rho']
ds['s_rho'].attrs["long_name"] = 'sigma'
ds['lon_rho'].attrs["long_name"] = 'lon_rho'
ds['lat_rho'].attrs["long_name"] = 'lat_rho'

### Modified Julian Date：MJD
mjd = '1858-11-17 00:00:00'
for s in ['ocean_time', 'zeta_time', 'v2d_time', 'v3d_time', 'salt_time', 'temp_time']:
    ds[s] = np.datetime64(mjd) + ds[s]

## Creating z coordinate values as `z_rho`
- See **Dictionary like methods** section in [data_structures](http://xarray.pydata.org/en/stable/data-structures.html)
- **Notice:** z_rho should be a function of time, which is ignored; consideration is required how to deal with. 

In [ ]:
def s_to_z_rho(ds, grd, itime):
    '''
    Creating z_rho (z coordinate value array at z_rho) using inver sigma-coordinate transform equation
    
    Args:
        ds  (xarray.Dataset) : Coordinates and variables
        grd (xarray.Dataset) : Grid
        itime (int) : Index value of time coordinate
    Returns:
        z_rho (ndarray) : z coordinate value array at (itime, s_rho[:], erho[:], xrho[:])
    '''

    z_rho = ds['zeta'][itime,:,:].values + \
    ds['s_rho'][0].values * (grd['h'].values + ds['zeta'][itime,:,:].values)
    #print(z_rho.shape)
    z_rho = z_rho[np.newaxis,:,:]  ### extend dimension
    #print(z_rho.shape)
    for k in np.arange(1, len(ds['s_rho'])):
        z_rho1 = ds['zeta'][itime,:,:].values + \
                 ds['s_rho'][k].values * (grd['h'].values + ds['zeta'][itime,:,:].values)
        z_rho1 = z_rho1[np.newaxis,:,:]
        #print(z_rho1.shape)
        z_rho = np.vstack((z_rho, z_rho1))
    return z_rho

In [ ]:
ds.coords['z_rho']=(('s_rho', 'erho', 'xrho'),s_to_z_rho(ds, grd, itime=0))
ds['z_rho'].attrs['long_name'] = 'Depth'
ds['z_rho'].attrs['units'] = 'm'
ds['z_rho'].attrs['field'] = 'depth, scalar, series'
ds

# Interactive plotting with [hvPlot](https://hvplot.holoviz.org/)
## Plot configuration
- To plot edge lines, `line_alpha` (0-1) should be a positive value.

In [ ]:
z='salt'
cmap='magma_r'
frame_height=300
frame_width=300
project = ccrs.PlateCarree()

## Plan view
- Set x-axis range: `xarray.Dataset.hvplot.quadmesh(xlim=(xmin, xmax))`
- Set num of color levels: `p.opts(opts.QuadMesh(color_levels=20))`
- Set value range: `xarray.Dataset.hvplot.quadmesh(clim=(cmin, cmax))`

In [ ]:
line_alpha = None  ###
clim = None  ### (cmin, cmax) or None
ds.hvplot.quadmesh(groupby=['s_rho','salt_time'], x='lon_rho', y='lat_rho',z=z , \
                   geo=True , project=project, tiles=None, coastline='10m', \
                   frame_height=frame_height, clim=clim, \
                   cmap=cmap, line_color='aqua', line_alpha=line_alpha, line_width=0.1).opts(
                   opts.QuadMesh(color_levels=20))

### Gets available options for hvplot.quadmesh()
- Plot options are specified by `quadmesh().opts()`

In [ ]:
hv.help(hv.QuadMesh)

## Vertical sectional views in sigma coordinate

In [ ]:
ds.hvplot.quadmesh(groupby=['xrho','salt_time'], x='erho', y='s_rho',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['erho','salt_time'], x='xrho', y='s_rho',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1)

## Vertical sectional view in z coordinate

In [ ]:
ds.hvplot.quadmesh(groupby=['xrho','salt_time'], x='erho', y='z_rho',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['erho','salt_time'], x='xrho', y='z_rho',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1).opts(bgcolor='lightgray')

# Plotting with matplotlib

## Plan view

In [ ]:
ds[z].isel(s_rho=0, salt_time=0).plot(x='lon_rho', y='lat_rho', extend='both', cmap='magma_r')

In [ ]:
p=ds[z].isel(s_rho=0, salt_time=0).plot(x='lon_rho', y='lat_rho', extend='both', cmap='magma_r',\
                                           vmin=31, vmax=36, alpha=1, edgecolor='aqua', linewidth=0.01)
p.axes.set_xlim(-77.5, -72.5)
p.axes.set_ylim(34, 38)
p.figure.savefig('temp.png', dpi=300, bbox_inches='tight')

## Vertical sectional view in sigma coordinate

In [ ]:
ds[z].isel(erho=0, salt_time=0).plot.pcolormesh(x='lon_rho', y='s_rho', extend='both', cmap='magma_r')

In [ ]:
ds[z].isel(xrho=0, salt_time=0).plot.pcolormesh(x='lat_rho', y='s_rho', extend='both', cmap='magma_r')

## Vertical sectional view in z coordinate

In [ ]:
ds[z].isel(erho=0, salt_time=0).plot.pcolormesh(x='lon_rho', y='z_rho', extend='both', cmap='magma_r')

In [ ]:
p=ds[z].isel(xrho=15, salt_time=0).plot.pcolormesh(x='lat_rho', y='z_rho', extend='both', \
                                                        cmap='magma_r')
p.axes.set_facecolor('lightgray')